**참고: https://untitledtblog.tistory.com/169**
- google cloud api 사이트 : https://console.cloud.google.com/apis/api/youtube.googleapis.com/credentials?project=skillful-cosine-311011&supportedpurview=project

In [ ]:
import pandas
from googleapiclient.discovery import build

In [ ]:
api_key = 'API Key를 입력'
video_id = '수집하고자 하는 비디오의 링크 뒷부분을 입력'
 
comments = list()
api_obj = build('youtube', 'v3', developerKey=api_key)
response = api_obj.commentThreads().list(part='snippet,replies', videoId=video_id, maxResults=1000).execute()
 
while response:
    for item in response['items']:
        comment = item['snippet']['topLevelComment']['snippet']
        comments.append([comment['textDisplay'], comment['authorDisplayName'], comment['publishedAt'], comment['likeCount']])
 
        if item['snippet']['totalReplyCount'] > 0:
            for reply_item in item['replies']['comments']:
                reply = reply_item['snippet']
                comments.append([reply['textDisplay'], reply['authorDisplayName'], reply['publishedAt'], reply['likeCount']])
 
    if 'nextPageToken' in response:
        response = api_obj.commentThreads().list(part='snippet,replies', videoId=video_id, 
                                                 pageToken=response['nextPageToken'], maxResults=100).execute()
    else:
        break
 
df = pandas.DataFrame(comments)
df.to_excel('youtube_total_comment.xlsx', header=['comment', 'author', 'date', 'num_likes'], index=None)